In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

import resnet

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

def getTrainData():
    X = []
    y = []

    for i in range(num_fold):
        fileX = os.path.join(base_data_path, 'X/X' + str(i+1) + '.npy')
        fileY = os.path.join(base_data_path, 'y/y' + str(i+1) + '.npy')
        
        X.append(np.load(fileX))
        y.append(np.load(fileY))

    X = np.array(X)
    y = np.array(y)

    return X, y

def split_data(X, y, idx):
    X_train = []
    y_train = []
    
    for i in range(num_fold):
        if i == idx:
            X_val = X[i]
            y_val = y[i]
            continue
        if X_train == []:
            X_train = X[i]
            y_train = y[i]
        else:
            X_train = np.concatenate((X_train, X[i]))
            y_train = np.concatenate((y_train, y[i]))

    return X_train, y_train, X_val, y_val

def normalize(X_train, X_val):
    X_train = (X_train - mean)/(std)
#     X_train = (X_train - min_)/range_
    X_val = (X_val - mean)/(std)
#     X_val = (X_val - min_)/range_

    return X_train, X_val


In [3]:

# min_ = np.swapaxes(min_,0,1)
# mean = np.swapaxes(mean,0,1)
# range_ = np.swapaxes(range_,0,1)
# std = np.swapaxes(std,0,1)

In [4]:
def train_valid(X_train,Y_train,X_valid,Y_valid,fold):
    model = [resnet.ResnetBuilder.build_resnet_18((1, X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_34((1,X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_50((1, X_train.shape[1], X_train.shape[2]), 41) ,
             resnet.ResnetBuilder.build_resnet_101((1, X_train.shape[1], X_train.shape[2]), 41),
             resnet.ResnetBuilder.build_resnet_152((1, X_train.shape[1], X_train.shape[2]), 41)]
    kk = random.randint(0, 4)
    model = model[kk]
    model.summary()
    if kk>2:
        batchSize=[32,64]#,128,256]
    elif kk==2:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64,128,256]
    
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    if not os.path.exists(saveD):
        os.makedirs(saveD)
    opt = Adam()#Nadam() #Adam(lr=2e-3,decay=1e-20)

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/mfcc6/'
    print('using resnet model: '+str(kk))
    if not os.path.exists(logD):
        os.makedirs(logD)
    history = History()
    callback=[
    #     ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(patien/2),
    #                                   min_lr=1e-4,mode='min', cooldown=1 ),
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_fold'+str(fold)+'_resnet'+str(kk)+'-.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,mode='max'),
        TensorBoard(log_dir=logD+'LGD_fold'+str(fold)+'_resnet'+str(kk)),
        history#,batch_size=batch_size, write_graph=True, write_grads=False, write_images=True)
    ]

    model.fit(X_train,Y_train,
              shuffle=True,
              callbacks=callback, 
              class_weight='auto',
              validation_data=(X_valid,Y_valid),
              batch_size=batchSize,
              epochs=epoch)
    return model,kk

In [5]:
def get_semi_data(X_train,Y_train):
    X_un_ver = np.load('feature/mfcc6/semi/fbank4/X_un_ver.npy')
    X_test_ver = np.load('feature/mfcc6/semi/fbank4/X_test_ver.npy')
    X = np.concatenate((X_un_ver,X_test_ver))
    Y_un_ver = np.load('feature/mfcc6/semi/fbank4/Y_un_ver.npy')
    Y_test_ver = np.load('feature/mfcc6/semi/fbank4/Y_test_ver.npy')
    Y = np.concatenate((Y_un_ver,Y_test_ver))
    Y = to_categorical(Y,num_classes=41)
    X_semi = np.concatenate((X_train,X))
    Y_semi = np.concatenate((Y_train,Y))
    X_semi , Y_semi = _shuffle(X_semi,Y_semi)
    print(X_semi.shape , Y_semi.shape)
    return X_semi , Y_semi

In [6]:
# model = resnet.ResnetBuilder.build_resnet_50((1, 64, 431), 41)
# model.summary()
def train_unverified(model,X_semi,Y_semi,fold,kk):
    name = glob.glob('model/'+feature_type+'/'+'LGD_fold'+str(fold)+'_resnet'+str(kk)+'-**')[0]
    print('semi loading: '+ name)
    model = load_model(name)
    if kk>2:
        batchSize=[32,64]#,128,256]
    elif kk==2:
        batchSize=[32,64,128]
    else:
        batchSize=[32,64,128,256]
    batchSize = random.choice(batchSize)
    patien=100
    epoch=3000
    saveD = 'model/'+feature_type+'/'
    opt = Adam(lr=0.0001,decay=1e-6)#Nadam() #Adam(lr=2e-3,decay=1e-20)

    model.compile(loss=['categorical_crossentropy'],optimizer=opt, metrics=['acc']) 
    logD = './logs/mfcc6/'
    history = History()
    callback=[
        ReduceLROnPlateau(monitor='loss', factor=0.2, patience=int(patien/20),min_lr=1e-6,
                          mode='min', cooldown=1,verbose=1 ),
        EarlyStopping(patience=patien,monitor='val_loss',verbose=1,
                      mode='min'),
        ModelCheckpoint(saveD+'LGD_semi_fold'+str(fold)+'_resnet'+str(kk)+'.h5',
                        monitor='val_acc',verbose=1,save_best_only=True, 
                        save_weights_only=False,
                        mode='max'),
        TensorBoard(log_dir=logD+'LGD_semi_fold'+str(fold)+'_resnet'+str(kk)),
        history
    ]

    model.fit(X_semi,Y_semi,
              shuffle=True,
              callbacks=callback, 
              class_weight='auto',
              validation_data=(X_valid,Y_valid),
              batch_size=batchSize,
              epochs=epoch)
    return model

In [7]:
feature_type = 'mfcc6'

In [8]:
mean = np.load('feature/'+feature_type+'/mean.npy')
std = np.load('feature/'+feature_type+'/std.npy')
min_ = np.load('feature/'+feature_type+'/min.npy')
range_ = np.load('feature/'+feature_type+'/range.npy')


base_path = 'feature/'+feature_type+'/'#'/tmp2/b03902110/newphase1'
base_data_path = 'feature/'+feature_type+'/'#os.path.join(base_path, 'data')
num_fold = 10

val_set_num = [2,3,4,5,6,7,8,9]#str(sys.argv[1])

In [ ]:
# fold=0
for fold in val_set_num:
    X, y = getTrainData()
    # X = np.swapaxes(X,2,3)
    X_train, Y_train, X_valid, Y_valid = split_data(X, y, fold) #fold
    # X_train, X_valid = normalize(X_train, X_valid)
    print(X_train.shape, Y_train.shape)

    # X_train = np.swapaxes(X_train,1,3)
    # X_valid = np.swapaxes(X_valid,1,3)
    print("===train verified_"+str(fold)+'===')
    model,model_num = train_valid(X_train,Y_train,X_valid,Y_valid,fold)
    X_semi , Y_semi = get_semi_data(X_train,Y_train)
    print('===train semi_'+str(fold)+'===')
    model_semi = train_unverified(model,X_semi,Y_semi,fold,model_num)
# 0=> 0.81941 (not semi)
# 1=>0.83019 (semi)
# 2=>0.81941 (semi)
# 3=>0.85984 (resnet1_not semi)、0.78437
# 4=>0.84367 (renet1_not semi)、0.81132
# 5=>

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


(3339, 64, 431, 1) (3339, 41)
===train verified_2===
using resnet model: 3
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 64, 431, 1)   0                                            
__________________________________________________________________________________________________
conv2d_110 (Conv2D)             (None, 32, 216, 64)  3200        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_103 (BatchN (None, 32, 216, 64)  256         conv2d_110[0][0]                 
__________________________________________________________________________________________________
activation_100 (Activation)     (None, 32, 216, 64)  0           batch_normalization_103[0][0]    
__________________________________

Train on 3339 samples, validate on 371 samples
Epoch 1/3000
3339/3339 [==============================] - 34s 10ms/step - loss: 13.1016 - acc: 0.2402 - val_loss: 13.1823 - val_acc: 0.1806

Epoch 00001: val_acc improved from -inf to 0.18059, saving model to model/mfcc6/LGD_fold2_resnet3-.h5
Epoch 2/3000
3339/3339 [==============================] - 20s 6ms/step - loss: 10.9475 - acc: 0.4028 - val_loss: 11.2230 - val_acc: 0.2372

Epoch 00002: val_acc improved from 0.18059 to 0.23720, saving model to model/mfcc6/LGD_fold2_resnet3-.h5
Epoch 3/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 9.1486 - acc: 0.5055 - val_loss: 8.8818 - val_acc: 0.3854

Epoch 00003: val_acc improved from 0.23720 to 0.38544, saving model to model/mfcc6/LGD_fold2_resnet3-.h5
Epoch 4/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 7.7450 - acc: 0.5627 - val_loss: 8.2251 - val_acc: 0.3827

Epoch 00004: val_acc did not improve from 0.38544
Epoch 5/3000
3339/3339 [==========


Epoch 00040: val_acc did not improve from 0.77358
Epoch 41/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 0.8898 - acc: 0.9536 - val_loss: 3.5644 - val_acc: 0.4933

Epoch 00041: val_acc did not improve from 0.77358
Epoch 42/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 0.9418 - acc: 0.9428 - val_loss: 2.1860 - val_acc: 0.6846

Epoch 00042: val_acc did not improve from 0.77358
Epoch 43/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 1.1336 - acc: 0.8958 - val_loss: 9.4080 - val_acc: 0.1833

Epoch 00043: val_acc did not improve from 0.77358
Epoch 44/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 1.0610 - acc: 0.9179 - val_loss: 2.7602 - val_acc: 0.6631

Epoch 00044: val_acc did not improve from 0.77358
Epoch 45/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 0.9209 - acc: 0.9512 - val_loss: 1.9580 - val_acc: 0.7035

Epoch 00045: val_acc did not improve from 0.77358
Epoc

3339/3339 [==============================] - 21s 6ms/step - loss: 0.6143 - acc: 0.9877 - val_loss: 1.6163 - val_acc: 0.7493

Epoch 00083: val_acc did not improve from 0.77898
Epoch 84/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 0.5952 - acc: 0.9889 - val_loss: 1.8689 - val_acc: 0.7251

Epoch 00084: val_acc did not improve from 0.77898
Epoch 85/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 0.5737 - acc: 0.9895 - val_loss: 1.5584 - val_acc: 0.7628

Epoch 00085: val_acc did not improve from 0.77898
Epoch 86/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 0.5200 - acc: 0.9970 - val_loss: 1.5436 - val_acc: 0.7601

Epoch 00086: val_acc did not improve from 0.77898
Epoch 87/3000
3339/3339 [==============================] - 21s 6ms/step - loss: 0.5038 - acc: 0.9973 - val_loss: 1.4791 - val_acc: 0.7736

Epoch 00087: val_acc did not improve from 0.77898
Epoch 88/3000
3339/3339 [==============================] - 21s 6ms/st

...

3339/3339 [==============================] - 26s 8ms/step - loss: 0.6882 - acc: 0.9467 - val_loss: 1.6420 - val_acc: 0.7439

Epoch 00083: val_acc did not improve from 0.79515
Epoch 84/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.6898 - acc: 0.9509 - val_loss: 1.8378 - val_acc: 0.7520

Epoch 00084: val_acc did not improve from 0.79515
Epoch 85/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.6363 - acc: 0.9644 - val_loss: 1.7653 - val_acc: 0.7655

Epoch 00085: val_acc did not improve from 0.79515
Epoch 86/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5994 - acc: 0.9695 - val_loss: 1.8493 - val_acc: 0.7493

Epoch 00086: val_acc did not improve from 0.79515
Epoch 87/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.6220 - acc: 0.9596 - val_loss: 1.9512 - val_acc: 0.6927

Epoch 00087: val_acc did not improve from 0.79515
Epoch 88/3000
3339/3339 [==============================] - 26s 8ms/st

3339/3339 [==============================] - 26s 8ms/step - loss: 0.5075 - acc: 0.9793 - val_loss: 1.7081 - val_acc: 0.7439

Epoch 00126: val_acc did not improve from 0.80593
Epoch 127/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.6496 - acc: 0.9437 - val_loss: 2.3301 - val_acc: 0.6065

Epoch 00127: val_acc did not improve from 0.80593
Epoch 128/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.6383 - acc: 0.9449 - val_loss: 2.0459 - val_acc: 0.7332

Epoch 00128: val_acc did not improve from 0.80593
Epoch 129/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5728 - acc: 0.9668 - val_loss: 1.9770 - val_acc: 0.7763

Epoch 00129: val_acc did not improve from 0.80593
Epoch 130/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5174 - acc: 0.9829 - val_loss: 1.6266 - val_acc: 0.7655

Epoch 00130: val_acc did not improve from 0.80593
Epoch 131/3000
3339/3339 [==============================] - 26s 8


Epoch 00168: val_acc did not improve from 0.81941
Epoch 169/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.6936 - acc: 0.9494 - val_loss: 2.2586 - val_acc: 0.6981

Epoch 00169: val_acc did not improve from 0.81941
Epoch 170/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5595 - acc: 0.9775 - val_loss: 1.6497 - val_acc: 0.7817

Epoch 00170: val_acc did not improve from 0.81941
Epoch 171/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5009 - acc: 0.9838 - val_loss: 1.6883 - val_acc: 0.7520

Epoch 00171: val_acc did not improve from 0.81941
Epoch 172/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.4708 - acc: 0.9892 - val_loss: 1.5850 - val_acc: 0.7951

Epoch 00172: val_acc did not improve from 0.81941
Epoch 173/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.4490 - acc: 0.9889 - val_loss: 1.4590 - val_acc: 0.8194

Epoch 00173: val_acc did not improve from 0.81941

3339/3339 [==============================] - 26s 8ms/step - loss: 0.5542 - acc: 0.9554 - val_loss: 2.7605 - val_acc: 0.5957

Epoch 00212: val_acc did not improve from 0.81941
Epoch 213/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5690 - acc: 0.9635 - val_loss: 3.0807 - val_acc: 0.6038

Epoch 00213: val_acc did not improve from 0.81941
Epoch 214/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5309 - acc: 0.9674 - val_loss: 1.8411 - val_acc: 0.7332

Epoch 00214: val_acc did not improve from 0.81941
Epoch 215/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5080 - acc: 0.9686 - val_loss: 1.8077 - val_acc: 0.7466

Epoch 00215: val_acc did not improve from 0.81941
Epoch 216/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5036 - acc: 0.9668 - val_loss: 2.3968 - val_acc: 0.6927

Epoch 00216: val_acc did not improve from 0.81941
Epoch 217/3000
3339/3339 [==============================] - 26s 8

3339/3339 [==============================] - 26s 8ms/step - loss: 0.3788 - acc: 0.9859 - val_loss: 1.7973 - val_acc: 0.7493

Epoch 00255: val_acc did not improve from 0.82749
Epoch 256/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.4489 - acc: 0.9635 - val_loss: 1.9986 - val_acc: 0.7224

Epoch 00256: val_acc did not improve from 0.82749
Epoch 257/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.4449 - acc: 0.9736 - val_loss: 1.4816 - val_acc: 0.7925

Epoch 00257: val_acc did not improve from 0.82749
Epoch 258/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.4779 - acc: 0.9662 - val_loss: 1.7589 - val_acc: 0.7574

Epoch 00258: val_acc did not improve from 0.82749
Epoch 259/3000
3339/3339 [==============================] - 26s 8ms/step - loss: 0.5630 - acc: 0.9497 - val_loss: 1.8421 - val_acc: 0.7493

Epoch 00259: val_acc did not improve from 0.82749
Epoch 260/3000
3339/3339 [==============================] - 26s 8